In [85]:
from bs4 import BeautifulSoup
from razdel import tokenize
import re
import gensim

In [2]:
def tokenization(text):
    tokens = list(tokenize(text.lower()))
    tokens = [_.text for _ in tokens]
    tokens_str = ' '.join(tokens)
    return tokens_str

In [3]:
with open('annot.opcorpora.xml', encoding='utf-8') as f:
    text = f.read()
    soup = BeautifulSoup(text, 'xml')
    clean_text = soup('source')
    clean_text = [tag.text for tag in clean_text]

with open('corpus.txt', 'w', encoding='utf-8') as f:
    for sentence in clean_text:
        f.write(tokenization(sentence) + '\n')

In [81]:
seed_markers = ['если', 'ежели', 'коли', 'коль', 'коль_скоро', 'кабы', 'когда_бы', 
                'в_случае_если', 'в_том_случае_если', 'в_случае_,_если', 'в_том_случае_,_если', 
                'при_условии_если', 'при_том_условии_если', 'при_условии_,_если', 'при_том_условии_,_если',
                'при_условии_что', 'при_том_условии_что', 'при_условии_,_что', 'при_том_условии_,_что']

pattern1 = re.compile('в ([а-я\-]+ )*случае( )*(\, )*([а-я\-]+)*')
pattern2 = re.compile('при ([а-я\-]+ )*условии( )*(\, )*([а-я\-]+)*')
pattern3 = re.compile('в ([а-я\-]+ )*случаях( )*(\, )*([а-я\-]+)*')
pattern4 = re.compile('при ([а-я\-]+ )*условиях( )*(\, )*([а-я\-]+)*')

after_comma = re.compile('\, ([а-я\-]+ ((если)|(когда)|(только)|(пока)))')

def regex_multitokens(text, pattern):
    
    while pattern.search(text) != None:
        result = pattern.search(text)
        space = result.group(0)
        no_space = re.sub(' ', '_', space)
        text = re.sub(space, no_space, text)
        
    return text

def seed_multitokens(text):
    
    text = text.replace('коль скоро', 'коль_скоро')
    text = text.replace('когда бы', 'когда_бы')
    
    return text

def pattern_multitokens(text):
    
    text = regex_multitokens(text, pattern1)
    text = regex_multitokens(text, pattern2)
    text = regex_multitokens(text, pattern3)
    text = regex_multitokens(text, pattern4)
    
    while after_comma.search(text) != None:
        result = after_comma.search(text)
        space = result.group(1)
        no_space = re.sub(' ', '_', space)
        text = re.sub(space, no_space, text)
    
    return text

In [82]:
stopwords = ['же', 'ж', 'ну', 'а', 'даже', 'лишь', 'и']

with open('corpus.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

clean_lines = []
for line in lines:
    words = line.split(' ')
    words = [word for word in words if word not in stopwords]
    if len(words) >= 2 and (words[1] == 'если' or words[1] == 'когда' 
        or words[1] == 'только' or words[1] == 'пока'):
        words[1] = str(words[0]) + '_' + str(words[1])
        words.pop(0)
    clean_lines.append(' '.join(words))
    

with open('corpus.txt', 'w', encoding='utf-8') as f:
    for line in clean_lines:
        f.write(line + '\n')
        
with open('corpus.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = seed_multitokens(text)
    text = pattern_multitokens(text)

with open('corpus_multi.txt', 'w', encoding='utf-8') as f:
    f.write(text)

In [105]:
data = gensim.models.word2vec.LineSentence('corpus_multi.txt')
model = gensim.models.Word2Vec(data, size = 200, sg=1, iter=10)
model.save('condition.model')

In [95]:
word_vectors = model.wv

In [109]:
candidates = {}
for marker in seed_markers:
    if marker in word_vectors.vocab:
        top20 = word_vectors.most_similar(positive = marker, topn = 20)
        candidates[marker] = [i[0] for i in top20]

for k, v in candidates.items():
    print(k)
    print(v)

если
['но_если', 'что_если', '«_если', 'ли', 'пусть', 'то', 'наверняка', 'в_том_случае_,_если', 'только_если', 'иностранец', 'вряд', 'вот_если', 'туристическая', 'если_только', 'виза', 'зачем', '•если', 'но_пока', 'фактически', 'иное']
ежели
['ну-ка', 'поди', 'игнатьевна', ':(', 'коли', 'клиневич', 'негодяй', 'сохрани', 'умею', 'умница', 'боже', 'давай', 'ух', 'слышите', 'я-то', 'желаете', 'братец', 'голубчик', 'душегубец', 'ей-богу']
коли
['братцы', 'ну-ка', 'ежели', '....', 'ей-богу', 'ванька', 'зобар', '=)', 'еле', 'умею', 'клиневич', 'ладно', 'послушай', 'девку', 'люблю', 'радде', ':(', 'лойко', 'игнатьевна', 'смотри']
коль
['приедете', 'роскомнадзору', 'предлагаете', 'управа', 'свершилось', 'набираетесь', 'пройдете', 'ваш-то', 'попенял', 'моментальный', 'спуску', 'стремитесь', 'отрицаете', 'ощущениям', 'называете', 'спекулировать', 'покушать', 'удосужились', 'машинисту', 'украинцам']
коль_скоро
['ляжем', 'субтитры', 'текстуры', 'худыми', 'вежливость', 'мумию', 'испортишь', 'мне-то

In [110]:
len(candidates.keys())

10

In [111]:
len(seed_markers)

19

In [112]:
for marker in seed_markers:
    if marker not in word_vectors.vocab:
        print(marker)

в_том_случае_если
в_случае_,_если
при_условии_если
при_том_условии_если
при_условии_,_если
при_том_условии_,_если
при_условии_что
при_том_условии_что
при_том_условии_,_что
